Azure Kubernetes 服务 (AKS) 是可用于快速部署和管理群集的托管式 Kubernetes 服务。此快速入门介绍如何：

- 使用 Azure CLI 部署 AKS 群集。
- 使用一组微服务和模拟零售场景的 Web 前端运行示例多容器应用程序。

> [!NOTE]
> 为了开始快速预配 AKS 群集，本文介绍了仅针对评估目的部署具有默认设置的群集的步骤。在部署生产就绪群集之前，建议熟悉我们的 [基线参考体系结构](https://learn.microsoft.com/zh-cn/azure/architecture/reference-architectures/containers/aks/baseline-aks?toc=/azure/aks/toc.json&bc=/azure/aks/breadcrumb/toc.json)，考虑它如何与你的业务需求保持一致。

## 开始之前

本快速入门假设读者基本了解 Kubernetes 的概念。有关详细信息，请参阅 [Azure Kubernetes 服务 (AKS) 的 Kubernetes 核心概念](https://learn.microsoft.com/zh-cn/azure/aks/concepts-clusters-workloads)。

- 如果没有 [Azure 订阅](https://learn.microsoft.com/zh-cn/azure/guides/developer/azure-developer-guide#understanding-accounts-subscriptions-and-billing)，请在开始之前创建一个 [Azure 免费帐户](https://azure.microsoft.com/free/?ref=microsoft.com&utm_source=microsoft.com&utm_medium=docs&utm_campaign=visualstudio)。
- 在 [Azure Cloud Shell](https://learn.microsoft.com/zh-cn/azure/cloud-shell/overview) 中使用 Bash 环境。有关详细信息，请参阅 [Azure Cloud Shell 中的 Bash 快速入门](https://learn.microsoft.com/zh-cn/azure/cloud-shell/quickstart)。
    
- 如需在本地运行 CLI 参考命令，请 [安装](https://learn.microsoft.com/zh-cn/cli/azure/install-azure-cli) Azure CLI。如果在 Windows 或 macOS 上运行，请考虑在 Docker 容器中运行 Azure CLI。有关详细信息，请参阅 [如何在 Docker 容器中运行 Azure CLI](https://learn.microsoft.com/zh-cn/cli/azure/run-azure-cli-docker)。
    
    - 如果使用的是本地安装，请使用 [az login](https://learn.microsoft.com/zh-cn/cli/azure/reference-index#az-login) 命令登录到 Azure CLI。若要完成身份验证过程，请遵循终端中显示的步骤。有关其他登录选项，请参阅 [使用 Azure CLI 登录](https://learn.microsoft.com/zh-cn/cli/azure/authenticate-azure-cli)。
        
    - 出现提示时，请在首次使用时安装 Azure CLI 扩展。有关扩展详细信息，请参阅 [使用 Azure CLI 的扩展](https://learn.microsoft.com/zh-cn/cli/azure/azure-cli-extensions-overview)。
        
    - 运行 [az version](https://learn.microsoft.com/zh-cn/cli/azure/reference-index?#az-version) 以查找安装的版本和依赖库。若要升级到最新版本，请运行 [az upgrade](https://learn.microsoft.com/zh-cn/cli/azure/reference-index?#az-upgrade)。
- 本文需要 Azure CLI 版本 2.0.64 或更高版本。如果你使用的是 Azure Cloud Shell，则表示已安装最新版本。
- 确保用于创建群集的标识具有合适的的最低权限。有关 AKS 访问和标识的详细信息，请参阅 [Azure Kubernetes Service (AKS) 的访问和标识选项](https://learn.microsoft.com/zh-cn/azure/aks/concepts-identity)。
- 如果有多个 Azure 订阅，请使用 [az account set](https://learn.microsoft.com/zh-cn/cli/azure/account#az-account-set) 命令选择应在其中计收资源费用的相应订阅 ID。有关详细信息，请参阅 [如何管理 Azure 订阅 – Azure CLI](https://learn.microsoft.com/zh-cn/cli/azure/manage-azure-subscriptions-azure-cli?tabs=bash#change-the-active-subscription)。



## 1.定义环境变量

定义在本快速入门中使用的以下环境变量：

In [ ]:
resource_group="test-aks-rg"
region="westeurope3"
cluster_name="test-aks-cluster"
dns_label="test-aks-cluster"

## 2.创建资源组

[Azure 资源组](https://learn.microsoft.com/zh-cn/azure/azure-resource-manager/management/overview) 是用于部署和管理 Azure 资源的逻辑组。 创建资源组时，系统会提示你指定一个位置。 此位置是资源组元数据的存储位置，也是资源在 Azure 中运行的位置（如果你在创建资源期间未指定其他区域）。

使用 [`az group create`](https://learn.microsoft.com/zh-cn/cli/azure/group#az-group-create) 命令创建资源组。

In [ ]:
!az group create --name {resource_group} --location {region}


## 3.创建 AKS 群集

使用 [`az aks create`](https://learn.microsoft.com/zh-cn/cli/azure/aks#az-aks-create) 命令创建 AKS 群集。以下示例使用一个节点创建一个群集，并启用系统分配的托管标识。

In [ ]:
!az aks create --resource-group {resource_group} --name {cluster_name} --node-count 1 --generate-ssh-keys


> [!NOTE]  
 > 当你创建新群集时，AKS 会自动创建第二个资源组来存储 AKS 资源。有关详细信息，请参阅 [为什么使用 AKS 创建两个资源组？](https://learn.microsoft.com/zh-cn/azure/aks/faq)

## 4.连接到群集

若要管理 Kubernetes 群集，请使用 Kubernetes 命令行客户端 [kubectl](https://kubernetes.io/docs/reference/kubectl/)。如果使用的是 Azure Cloud Shell，则 `kubectl` 已安装。若要在本地安装 `kubectl`，请使用 [`az aks install-cli`](https://learn.microsoft.com/zh-cn/cli/azure/aks#az-aks-install-cli) 命令。

1. 使用 [az aks get-credentials](https://learn.microsoft.com/zh-cn/cli/azure/aks#az-aks-get-credentials) 命令将 `kubectl` 配置为连接到你的 Kubernetes 群集。此命令将下载凭据，并将 Kubernetes CLI 配置为使用这些凭据。

In [ ]:
!az aks get-credentials --resource-group {resource_group} --name {cluster_name}

2. 使用 [kubectl get](https://kubernetes.io/docs/reference/generated/kubectl/kubectl-commands#get) 命令验证与群集之间的连接。此命令将返回群集节点的列表。

In [ ]:
!kubectl get nodes